In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

## 1. Load Data

In [9]:
base_dir  = '../input/rock-paper-scissors-dataset/Rock-Paper-Scissors/'
train_dir = base_dir + 'train/'
test_dir  = base_dir + 'test/'

## 2. Prepare Data

In [12]:
train_datagen_augmented = ImageDataGenerator(
    rescale          = 1./255,
    rotation_range   = 20,
    horizontal_flip  = True,
    shear_range      = 0.2,
    fill_mode        = 'nearest',
    validation_split = 0.2
)
test_data = ImageDataGenerator(
    rescale = 1./255
)

In [13]:
train_data_augmented = train_datagen_augmented.flow_from_directory(
    directory   = train_dir,
    target_size = (150, 150),
    batch_size  = 32,
    class_mode  = 'categorical',
    subset      = 'training',
    shuffle     = True
)
test_data = test_data.flow_from_directory(
    directory   = test_dir,
    target_size = (150, 150),
    batch_size  = 1,
    class_mode  = 'categorical',
    shuffle     = True
)

FileNotFoundError: [Errno 2] No such file or directory: '../input/rock-paper-scissors-dataset/Rock-Paper-Scissors/train/'

## 3. Create Model

In [4]:
model = Sequential([
    
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Dropout(0.2),
    
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Dropout(0.2),
    
    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Dropout(0.2),
    
    Conv2D(256, (3,3), activation='relu'),
    MaxPooling2D((2,2)),
    BatchNormalization(),
    Dropout(0.2),
    
    Flatten(),
    
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.2),
    
    Dense(1, activation='sigmoid')
    
])

2023-11-16 12:02:02.306431: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:880] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-11-16 12:02:02.378461: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [6]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

In [7]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True
)

In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 74, 74, 32)        0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 74, 74, 32)        128       
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 74, 74, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 36, 36, 64)        0

## 4. Train and Evaluate Model

In [11]:
histories = []

In [ ]:
history = model.fit(
    train_data_augmented,
    epochs          = 2000,
    steps_per_epoch = len(train_data_augmented),
    validation_data = test_data,
    validation_steps= len(test_data),
    callbacks       = [early_stopping]
)
history.append(history.history)